# **Exploração Visual das Correspondências de Cândido Portinari através de Metadados Textuais**
#### **por Micaele Brandão e Mateus Bandeira**

### **Tópicos**
1. Introdução
2. Configuração de ambiente
3. Coleta e limpeza de dados  
4. Cacacterísticas do DataFrame final 
5. Visualizações  
.   
6. Conclusão e Referências


# 1. Introdução

Este projeto se trata de uma exploração visual das corresponências de Cândido Portinari, pintor brasileiro e um dos principais nomes do Modernismo. Um artista que, como disse Israel Pedrosa:  “Nenhum pintor pintou mais um País do que Portinari pintou o seu”. Nesse sentido, por meio do processamento de metadados textuais, buscamos identificar padrões temáticos nas comunicações do artista, mapear sua extensa rede de interlocutores e contextualizar cronologicamente os principais marcos de sua carreira. Para isso, utilizamos técnicas de web scraping, um método de coleta automatizada que permite extrair grandes volumes de dados de plataformas digitais, convertendo registros dispersos em uma base estruturada. O objetivo central é transformar esses documentos históricos em insights visuais que facilitem a compreensão da trajetória pessoal e profissional do artista. Ao transformar correspondências em representações gráficas, o projeto permite visualizar de forma intuitiva a circulação de Portinari no cenário artístico global, evidenciando como sua produção e seus diálogos ajudaram a moldar a identidade cultural do Brasil. 

O repositório deste projeto está disponível no GitHub através do link: [Repositório portinari-metadata](https://github.com/vrsmic/portinari-metadata/tree/main)


# 2. Configuração de ambiente

In [2]:
import pandas as pd
import altair as alt

#scrapping 1
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

#scrapping 2
import asyncio
import random

# apenas para fazer o scrapping, necessita de comandos no terminal.
#from playwright.async_api import async_playwright
#from tqdm.notebook import tqdm

import re
import warnings
# configuração opcional para suprimir avisos de concatenação futura, se desejar
warnings.simplefilter(action='ignore', category=FutureWarning)


# 3. Coleta e limpeza de dados

## 3.1. Coleta dos dados
Códigos para fazer o web scrapping do site com as correspondências de Portinari

## 3.2. Definição de Funções Auxiliares
Para manter o código limpo, definimos aqui as funções que serão reutilizadas ao longo do notebook:
1. `processar_e_juntar`: Automatiza a leitura, padronização e concatenação dos arquivos csv.
2. `extrair_campo`: Aplica expressões regulares para extrair metadados do texto bruto.

In [3]:
def processar_e_juntar(df_acumulado, arquivo_novo):
    """
    lê um novo arquivo csv, padroniza as colunas e junta com o dataframe acumulado
    """
    print(f"\n-- Processando junção com: {arquivo_novo} --")
    
    # carregar os arquivos
    try:
        # 'on_bad_lines' pra pular linhas com erro se houver, e encoding utf-8
        df_extraidos = pd.read_csv(arquivo_novo)
    except FileNotFoundError:
        print(f"Erro: Arquivo {arquivo_novo} não encontrado.")
        return df_acumulado

    # ajustar 'dados_extraidos' 
    # o arquivo extraído tem muitas colunas vazias no final, vamos pegar apenas as úteis
    cols_uteis = ['titulo', 'conteudo']
    # Verificação simples se as colunas existem
    if not all(col in df_extraidos.columns for col in cols_uteis):
        print(f"Colunas necessárias ausentes em {arquivo_novo}. Pulando.")
        return df_acumulado
        
    df_extraidos = df_extraidos[cols_uteis].copy()

    # adc as colunas que faltam para ficar no padrão do docvirt
    # se não houver informação, deixamos como None ou um texto padrão
    df_extraidos['pagina'] = None 
    df_extraidos['status'] = 'extraido_novo' 

    # reordenar as colunas do extraído para ficarem na mesma ordem do docvirt
    colunas_ordem = df_acumulado.columns.tolist()
    for col in colunas_ordem:
        if col not in df_extraidos.columns:
            df_extraidos[col] = None
    df_extraidos = df_extraidos[colunas_ordem]

    # juntar e completar
    # concatenamos os dois e colocamos o docvirt primeiro, pois ele parece mais completo 
    df_final = pd.concat([df_acumulado, df_extraidos], ignore_index=True)

    # remover duplicatas
    # se o mesmo 'titulo' existir nos dois, mantemos o primeiro (que veio do docvirt)
    # isso garante que completamos a lista sem criar linhas repetidas
    df_final = df_final.drop_duplicates(subset=['titulo'], keep='first')
    
    print(f"Total de linhas após juntar: {len(df_final)}")
    return df_final

def extrair_campo(texto, padrao):
    """ função para extrair padrões do texto cru usando regex """
    if pd.isna(texto):
        return None
    match = re.search(padrao, texto, re.MULTILINE | re.IGNORECASE)
    return match.group(1).strip() if match else None

## 3.3. Consolidação dos Dados
Carregamos a base principal e iteramos sobre os arquivos novos para criar um único dataframe consolidado.

In [4]:
# arquivo base original
arquivo_base = 'Data/dados_docvirt.csv'
df_final = pd.read_csv(arquivo_base)
print(f"Base inicial carregada ({arquivo_base}). Linhas: {len(df_final)}")

# lista de arquivos para adicionar sequencialmente
arquivos_novos = [
    'Data/dados_extraidos.csv',
    'Data/dados_extraidos2.csv',
    'Data/dados_extraidos3.csv'
]

# loop de processamento
for arquivo in arquivos_novos:
    df_final = processar_e_juntar(df_final, arquivo)

# salva um checkpoint consolidado 
df_final.to_csv('Data/dados_consolidados_bruto.csv', index=False)
display(df_final.head())

Base inicial carregada (Data/dados_docvirt.csv). Linhas: 9126

-- Processando junção com: Data/dados_extraidos.csv --
Total de linhas após juntar: 5803

-- Processando junção com: Data/dados_extraidos2.csv --
Total de linhas após juntar: 5803

-- Processando junção com: Data/dados_extraidos3.csv --
Total de linhas após juntar: 5836


,pagina,titulo,conteudo,status
0,1,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\r\nNUM_ANTDOC: 1\r\nPAGINAÇÃO: \r...,sucesso
1,2,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\r\nNUM_ANTDOC: 2\r\nPAGINAÇÃO: \r...,sucesso
2,3,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\r\nNUM_ANTDOC: 3\r\nPAGINAÇÃO: \r...,sucesso
3,4,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\r\nNUM_ANTDOC: 4\r\nPAGINAÇÃO: \r...,sucesso
4,5,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\r\nNUM_ANTDOC: 5\r\nPAGINAÇÃO: \r...,sucesso


## 3.4. Padronização e Verificação das Colunas de Identificação
Realizamos a limpeza básica nos tipos de dados e strings:

1. Criação da coluna `id_doc` limpa;
2. Conversão da coluna `pagina` para numérico;
3. Remoção de espaços em branco;
4. Remoção de dados pegos não intencionalmente;
5. Verificação dos ID's de documentos que não existem ou estão sem metadados ou com formatação errada.


In [5]:
# extrair um ID limpo do título antes de filtrar qualquer coisa
df_final['id_doc'] = df_final['titulo'].astype(str).str.extract(r'\\([^\\\\]+)$')

ids_esperados = {f'CO_{str(i).zfill(4)}' for i in range(1, 5893)}
ids_presentes_brutos = set(df_final['id_doc'].dropna().unique())
ids_realmente_faltantes = ids_esperados - ids_presentes_brutos

print(f"Total de correspondências na numeração (1 a 5892): {len(ids_esperados)}")
print(f"Qtd. IDs encontrados no site: {len(ids_presentes_brutos)}")
print(f"Qtd. IDs que não existem ou não são úteis: {len(ids_realmente_faltantes)}") 

print(f"ID's que não existem/úteis: {list(sorted(ids_realmente_faltantes))}\n")

# retirando vazios e erros
# verifica quem está vazio ou não é do tipo 'CO' válido antes de deletar
mask_vazios = (df_final['conteudo'].isna()) | (df_final['conteudo'] == '') | (~df_final['conteudo'].fillna('').str.startswith('COD_TIPO: CO'))


print(f"IDs presentes mas sem conteúdo válido (Vazios/Imagens/Erro): {mask_vazios.sum()}")

# limpa o dataframe final removendo os vazios e erros
df_final = df_final[~mask_vazios].copy()

# tratamentos estéticos restantes
df_final['pagina'] = pd.to_numeric(df_final['pagina'], errors='coerce').astype('Int64')
cols_texto = df_final.select_dtypes(include=['object']).columns
for col in cols_texto:
    df_final[col] = df_final[col].astype(str).str.strip().replace({'nan': None, 'None': None, '': None})

print(f"\nTotal final para análise (com texto válido): {len(df_final)}")
display(df_final.head())

Total de correspondências na numeração (1 a 5892): 5892
Qtd. IDs encontrados no site: 5835
Qtd. IDs que não existem ou não são úteis: 73
ID's que não existem/úteis: ['CO_1807', 'CO_1875', 'CO_2043', 'CO_2095', 'CO_2192', 'CO_2769', 'CO_3027', 'CO_3028', 'CO_3029', 'CO_3037', 'CO_3061', 'CO_3062', 'CO_3066', 'CO_3083', 'CO_3109', 'CO_3123', 'CO_3125', 'CO_3136', 'CO_3164', 'CO_3165', 'CO_3166', 'CO_3167', 'CO_3185', 'CO_3208', 'CO_3439', 'CO_3506', 'CO_3629', 'CO_3674', 'CO_3890', 'CO_4043', 'CO_4047', 'CO_4048', 'CO_4050', 'CO_4051', 'CO_4053', 'CO_4057', 'CO_4063', 'CO_4064', 'CO_4087', 'CO_4098', 'CO_4099', 'CO_4101', 'CO_4116', 'CO_4251', 'CO_4306', 'CO_4307', 'CO_4308', 'CO_4310', 'CO_4311', 'CO_4312', 'CO_4313', 'CO_4320', 'CO_4328', 'CO_4344', 'CO_4353', 'CO_4394', 'CO_4395', 'CO_4405', 'CO_4406', 'CO_4409', 'CO_4412', 'CO_4416', 'CO_4420', 'CO_4681', 'CO_4686', 'CO_4689', 'CO_4690', 'CO_5190', 'CO_5308', 'CO_5318', 'CO_5431', 'CO_5539', 'CO_5673']

IDs presentes mas sem conteúdo

,pagina,titulo,conteudo,status,id_doc
0,1,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\r\nNUM_ANTDOC: 1\r\nPAGINAÇÃO: \r...,sucesso,CO_0001
1,2,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\r\nNUM_ANTDOC: 2\r\nPAGINAÇÃO: \r...,sucesso,CO_0002
2,3,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\r\nNUM_ANTDOC: 3\r\nPAGINAÇÃO: \r...,sucesso,CO_0003
3,4,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\r\nNUM_ANTDOC: 4\r\nPAGINAÇÃO: \r...,sucesso,CO_0004
4,5,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\r\nNUM_ANTDOC: 5\r\nPAGINAÇÃO: \r...,sucesso,CO_0005


## 3.5. Extração de Metadados 
Usamos as expressões regulares definidas para estruturar o texto da coluna `conteudo` em novas colunas (Remetente, Destinatário, Data, Tipo, Onomástico, Memo_Resumo).

In [6]:
# padrões de regex baseados no formato do seu arquivo
padroes = {
    'remetente': r'REMETENTE:\s*(.*?)(?:\n|$)',
    'destinatario': r'DESTINATARIO:\s*(.*?)(?:\n|$)',
    'data': r'DATA_INICIO:\s*(.*?)(?:\n|$)',
    'tipo_doc': r'TIPO CORRESPONDÊNCIA:\s*(.*?)(?:\n|$)',
    
    # Novos campos adicionados
    # O regex captura o texto logo após os dois pontos até o fim da linha
    'onomastico': r'ONOMÁSTICO:\s*(.*?)(?:\n|$)',
    'memo_resumo': r'(?:RESUMO|MEMO_RESUMO):\s*(.*?)(?:\n|$)' # Tenta pegar RESUMO ou MEMO_RESUMO
}

# aplica a extração para criar novas colunas usando a função definida no passo anterior
for nova_coluna, regex in padroes.items():
    df_final[nova_coluna] = df_final['conteudo'].apply(lambda x: extrair_campo(str(x), regex))

# visualizar o resultado das novas colunas
colunas_visualizar = ['id_doc'] + list(padroes.keys())
display(df_final[colunas_visualizar].head())

,id_doc,remetente,destinatario,data,tipo_doc,onomastico,memo_resumo
0,CO_0001,Iris Abbott,Candido Portinari,1940/11/08,Telegrama,null,"Convida Portinari para jantar, destacando pres..."
1,CO_0002,Júlio Jorge Abeid Filho,Candido Portinari,1956/10/04,Carta,Oswaldo Scatena,Acusa o recebimento e agradece a doação da tel...
2,CO_0003,Lívio Abramo,Candido Portinari,1945/11/26,Carta,Marques Rebelo,Solidariza-se com Portinari pela não realizaçã...
3,CO_0004,Tharcema Cunha de Abreu,Candido Portinari,1946/ /,Carta,null,Pede que Portinari envie convites (para sua mo...
4,CO_0005,Aníbal Freire,Júlio Prestes,1926/06/05,Carta,null,Carta de recomendação ao deputado Júlio Preste...


## 3.6. Limpeza Final
Realizamos os últimos filtros e ajustes antes de salvar o arquivo final:
1. Ordenar pelo ID.
2. Preencher páginas vazias usando a lógica sequencial (`ffill`).

In [7]:
# ordena o df por CO_NNNN crescentemente usando a coluna id_doc 
df_final = df_final.sort_values(by='id_doc')

# completa a coluna página, substituindo os <NA> pelo valor correto ordenado, de acordo com a posição da linha no df
# o ffill propaga o último valor válido para frente, como ordenamos por ID, isso preenche páginas sequenciais.
df_final['pagina'] = df_final['pagina'].ffill()

# apagar a coluna 'status' que era só para controle interno, não é mais necessária
df_final = df_final.drop(columns=['status'], errors='ignore')

# apagar a coluna 'conteudo' que foi distribuidora das informações para as novas colunas, não é mais necessária
df_final = df_final.drop(columns=['conteudo'], errors='ignore')

# apagar a coluna 'titulo' que era só para controle interno, não é mais necessária
df_final = df_final.drop(columns=['titulo'], errors='ignore')

# ajustando a coluna data de AAAA/MM/DD para DD/MM/AAAA, sem apagar os que possuem data incompleta
# função que bota o que está antes do primeiro '/' no final, e o que está depois do segundo '/' no começo, mantendo o que está entre os dois no meio
def ajustar_data(data):
    if pd.isna(data):
        return None
    partes = data.split('/')
    if len(partes) == 3:
        return f"{partes[2]}/{partes[1]}/{partes[0]}"
    return data  # retorna a data original se não estiver no formato esperado
df_final['data'] = df_final['data'].apply(ajustar_data)


# teste com o CO_0502, que estava com página vazia no seu exemplo original
print("Verificação do CO_0502:")
display(df_final[df_final['id_doc'] == 'CO_0502'])

Verificação do CO_0502:


,pagina,id_doc,remetente,destinatario,data,tipo_doc,onomastico,memo_resumo
5805,803,CO_0502,Mário Dionísio,Candido Portinari,27/11/1952,Carta,J. J. C.,"Informa que, tão logo recebeu recorte do ""Corr..."


## 3.7 Salvamento do CSV

In [8]:
# salvar como dados finais
nome_arquivo_final = 'Data/dados_finais.csv'
df_final.to_csv(nome_arquivo_final, index=False)
print(f"Arquivo '{nome_arquivo_final}' salvo")

Arquivo 'Data/dados_finais.csv' salvo


# 4. Cacacterísticas do DataFrame final

In [9]:
df = pd.read_csv('Data/dados_finais.csv')
df

,pagina,id_doc,remetente,destinatario,data,tipo_doc,onomastico,memo_resumo
0,1,CO_0001,Iris Abbott,Candido Portinari,08/11/1940,Telegrama,NaN,"Convida Portinari para jantar, destacando pres..."
1,2,CO_0002,Júlio Jorge Abeid Filho,Candido Portinari,04/10/1956,Carta,Oswaldo Scatena,Acusa o recebimento e agradece a doação da tel...
2,3,CO_0003,Lívio Abramo,Candido Portinari,26/11/1945,Carta,Marques Rebelo,Solidariza-se com Portinari pela não realizaçã...
3,4,CO_0004,Tharcema Cunha de Abreu,Candido Portinari,/ /1946,Carta,NaN,Pede que Portinari envie convites (para sua mo...
4,5,CO_0005,Aníbal Freire,Júlio Prestes,05/06/1926,Carta,NaN,Carta de recomendação ao deputado Júlio Preste...
...,...,...,...,...,...,...,...,...
4142,9122,CO_5888,Abraham Mandelstam,Candido Portinari,28/11/1941,Carta,NaN,Pede permissão para incluir o nome de Portinar...
4143,9123,CO_5889,Candido Portinari,Olegário Mariano,/07/1930,Carta,NaN,Rafirma o afeto que o liga a Olegário.
4144,9124,CO_5890,Rodolfo Lima Martensen,Candido Portinari,11/02/1952,Carta,NaN,Mostra-se preocupado que Portinari não tenha r...
4145,9125,CO_5891,Renato Firmino Maia de Mendonça,Guillermo Winter,18/05/1959,Carta,Aguinaldo Boulitreau Fragoso,Informa estar enviando cópia de telegrama rece...


In [10]:
print("Colunas do df final:")
print(df.columns.tolist())

# info
print("\nInformações do df:")
df.info()

# describe
'''print("\nDescrição estatística do df:")
df.describe(include='all')'''

Colunas do df final:
['pagina', 'id_doc', 'remetente', 'destinatario', 'data', 'tipo_doc', 'onomastico', 'memo_resumo']

Informações do df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4147 entries, 0 to 4146
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   pagina        4147 non-null   int64 
 1   id_doc        4147 non-null   object
 2   remetente     4017 non-null   object
 3   destinatario  4094 non-null   object
 4   data          4140 non-null   object
 5   tipo_doc      4147 non-null   object
 6   onomastico    1706 non-null   object
 7   memo_resumo   4147 non-null   object
dtypes: int64(1), object(7)
memory usage: 259.3+ KB


'print("\nDescrição estatística do df:")\ndf.describe(include=\'all\')'

In [11]:
# verificar onde existem valores nulos ou vazios
print("\nContagem de valores nulos por coluna:")
print(df.isnull().sum())


Contagem de valores nulos por coluna:
pagina             0
id_doc             0
remetente        130
destinatario      53
data               7
tipo_doc           0
onomastico      2441
memo_resumo        0
dtype: int64


# 5. Visualizações

## 5.1 Paleta de Cores das Visualizações
Análise da Obra ‘Bandeirantes’ de Portinari.


<!-- Inserção de imagem 'Paleta.png' -->
![Paleta de Cores](Images/Paleta.png)


In [ ]:
# definição das cores
cores = {
    'Azul': '#4F6F8C',
    'Dourado': '#BF8821',
    'Bege': '#D9C2AD',
    'Vermelho': '#A62014',
    'Vinho': '#59150E',
    'Cinza': "#6E6E6E",
    'Branco': "#FFFFFF",
    'Preto': '#000000'
}


## 5.2 Análise dos tipos de correspondência


In [17]:
# quais tipos existem
tipos = df['tipo_doc']
print("\nTipos de documentos presentes:")
print(tipos.value_counts())


Tipos de documentos presentes:
tipo_doc
Carta             3312
Telegrama          355
Bilhete            182
Cartão             162
Cartão-postal       91
Carta Circular      19
Bilhete-postal      10
Memorando            5
Declaração           5
Comunicado           4
Saudação             1
Dedicatória          1
Name: count, dtype: int64


In [23]:
import pandas as pd
import altair as alt

# 1. Carregar os dados
df = pd.read_csv('Data/dados_finais.csv')

# 2. Preparar os dados (contagem por tipo)
tipo_counts = df['tipo_doc'].value_counts().reset_index()
tipo_counts.columns = ['tipo_doc', 'quantidade']

# 3. Definir sua paleta de cores personalizada
cores_portinari = ['#4F6F8C', '#BF8821', '#D9C2AD', '#A62014', '#59150E']

# 4. Criar o gráfico no Altair
chart = alt.Chart(tipo_counts).mark_bar().encode(
    # Eixo X ordenado pela quantidade (descendente)
    x=alt.X('tipo_doc:N', 
            sort='-y', 
            title='Tipo de Documento',
            axis=alt.Axis(labelAngle=-45)),
    
    # Eixo Y com a contagem
    y=alt.Y('quantidade:Q', 
            title='Total de Documentos'),
    
    # Cores baseadas na sua paleta
    color=alt.Color('tipo_doc:N', 
                    scale=alt.Scale(range=cores_portinari), 
                    legend=None),
    
    # Tooltip interativo (aparece ao passar o mouse)
    tooltip=['tipo_doc', 'quantidade']
).properties(
    title='Distribuição da Correspondência de Portinari por Tipo',
    width=600,
    height=400
).configure_title(
    fontSize=18,
    font='Arial',
    anchor='start',
    color='#59150E' # Usando o Vinho da sua paleta no título
).configure_view(
    strokeWidth=0 # Remove a borda ao redor do gráfico
)

# 5. Exibir o gráfico
chart.show()
# Se estiver no Jupyter/VS Code, basta digitar apenas 'chart' na última linha

alt.Chart(...)